# Anwendungsübungen zum RSA Algorithmus

Erstellen Sie ein Schlüsselpaar für die Verschlüsselung mit RSA, mit dem Sie
dreistellige Zahlen verschlüsseln können.

```{admonition} Mögliche Lösung
:class: tip, dropdown
Damit eine dreistellige Zahl verschlüsselt werden kann, muss das RSA-Modul
grösser als 1'000 sein. Es müssen also zwei Primzahlen $p$ und $q$ gefunden
werden, die etwas grösser als $\sqrt{1000}$ sind. Eine Hilfe für das Finden von
Primzahlen ist zum Beispiel die Seite
"[Die
Primzahlenseite](https://www.arndt-bruenner.de/mathe/scripts/primzahlen.htm)".

Für das Beispiel werden die Primzahlen $p = 37$ und $q = 41$ gewählt. Das RSA-Modul
ist dann $n = p \cdot q = 37 \cdot 41 = 1517$.  
Für die Verschlüsselung wird der öffentliche Exponent $e = 17$ gewählt.

Für die Berechnung des privaten Exponenten $d$ wird der erweiterte Euklidsche
Algorithmus verwendet. Dazu wird in einer ersten Phase der $ggT((p-1) \cdot
(q-1), e)$ berechnet. In einer zweiten Phase wird $s$ aus der Gleichung $ggT((p-1) \cdot
(q-1), e) = p \cdot (p-1) \cdot (q-1) + s \cdot e$ als $d$ berechnet.

Phase 1:

$$
\begin{aligned}
a&=q \cdot b + r\\
1440&=84 \cdot 17 + 12\\
17&=1 \cdot 12 + 5\\
12&=2 \cdot 2 + 1\\
2&=2 \cdot 1 + 0\\
\end{aligned}
$$

Phase 2:

$$
\begin{alignat*}{3}
&S_0 &&= 0&&\\
&S_1 &&= 1&&\\
&S_2 &&= S_0 - q_1 \cdot S_1 = 0 - 84 \cdot 1 &&= -84\\
&S_3 &&= S_1 - q_2 \cdot S_2 = 1 - 1 \cdot (-84) &&= 85\\
&S_4 &&= S_2 - q_3 \cdot S_3 = -84 - 2 \cdot 85 &&= -254\\
&S_5 &&= S_3 - q_4 \cdot S_4 = 85 - 2 \cdot (-254) &&= 593\\
\end{alignat*}
$$

Der private Exponent ist $d = 593$.
```

In [1]:
def erweiterter_euklid(a, b):
    # Basisfall: wenn a = 0
    if a == 0:
        return b, 0, 1
    
    # Rekursiver Aufruf
    gcd, x1, y1 = erweiterter_euklid(b % a, a)
    
    # Berechnung der Koeffizienten x und y
    x = y1 - (b // a) * x1
    y = x1
    
    return gcd, x, y

In [2]:
def decryptor(a: int, b: int) -> int:
    gcd, x, y = erweiterter_euklid(a, b)
    if y < 0:
        return y + a
    else:
        return y

In [3]:
decryptor((36*40), 17)

593